# Проект для Благотворительного фонда АНО Рак победим

###Задачи:
* предобработка данных;
* построение дашбордов. Дашборд "Подопечные" отражает географию подопечных фонда, диагноз, стадии лечения и т.д. Дашборд "Доноры" включает в себя аналитику доноров и пожертвований фонда.

### Ссылка на дашборды: https://datalens.yandex/fg30id4a9lsm3

### Ссылка на сайт фонда: https://rak-pobedim.com/

## 1. Подготовка к анализу

In [ ]:
pip install yadisk

In [ ]:
pip install fuzzywuzzy

In [ ]:
pip install python-Levenshtein

In [ ]:
pip install python-dotenv

In [ ]:
pip install roman

### 1.1 Загрузка библиотек

In [ ]:
# Standard Library imports
import warnings

# Data Science
import pandas as pd

# Math
import scipy.stats as st
import numpy as np
import roman

#Text
import re
import string
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import graph_objects as go
import seaborn as sns

import yadisk
from openpyxl import load_workbook
from google.colab import files
from google.colab import userdata

### 1.2 Написание функций

In [ ]:
# функция для изучения датасета
def describe_df(df):
    print("\33[1m"'Общая информация о датасете'"\033[0m")
    display(df.info())

    print("\33[1m"'% пропусков по столбцам'"\33[0m")
    display(round(df.isna().sum()/df.shape[0]*100, 2))

    print("\33[1m"'Количество дубликатов в датасете'"\33[0m")
    display(df.duplicated().sum())

    print("\33[1m"'Доля дубликатов в датасете'"\33[0m")
    display(round(df.duplicated().sum()/df.shape[0] *100,2))

In [ ]:
 # для замены значения согласно словарю
def find_in_dict(dic, text, changing):
    #Перибираем элементы словаря, для каждого ключа проверяем каждое из значений в списке
    for key in dic:
        #Если совпадание найдено, присваиваем переменной соответственное значение ключа, останавливаем перебор, переходим к следующей строке
        if text in key:
            return dic[key]
    #Если нет, то заданное значение
    return changing

In [ ]:
# для замены согласно словаю с поиском совпадений
def find_substring(x, dict, changing):
  #Начальное значение ключа - если подстрока не найдется, то будет заданное значение
    found_key = changing
    #Перибираем элементы словаря, для каждого ключа проверяем каждое из значений в списке
    for key, values in dict.items():
        for value in values:
          #Ищем любое совпадение - если совпадание найдено, присваиваем переменной соответственное значение ключа, останавливаем перебор, переходим к следующей строке
            if pd.Series(x).astype(str).str.contains(value, regex=True, case=False).any():
                found_key = key
                break
    #Функция возвращает значение переменной found_key - если совпадание нашлось, то это будет название категории, если не нашлось - то заданное
    return found_key

### 1.3 Загрузка данных

In [ ]:
from google.colab import userdata
app_id = userdata.get('app_id')

In [ ]:
from google.colab import userdata
secret_id = userdata.get('secret_id')

In [ ]:
from google.colab import userdata
ya_token = userdata.get('ya_token')

In [ ]:
y = yadisk.YaDisk(app_id, secret_id, ya_token)
y.check_token()

True

In [ ]:
for el in list(y.listdir('dataset')):
    if el['path'].endswith('.xlsx'):
        print(el['path'])

disk:/dataset/Pак_победим_доноры.xlsx
disk:/dataset/Анкеты_до_30_11_2023_new_data_1.xlsx
disk:/dataset/Справочники для проекта Рак Победим.xlsx


In [ ]:
y.download('/dataset/Pак_победим_доноры.xlsx', 'Pак_победим_доноры.xlsx')
donors = pd.read_excel('Pак_победим_доноры.xlsx', parse_dates = ['Дата пожертвования', 'Date '])

y.download('/dataset/Анкеты_до_30_11_2023_new_data_1.xlsx', 'Анкеты_до_30_11_2023_new_data_1.xlsx')
wards = pd.read_excel('Анкеты_до_30_11_2023_new_data_1.xlsx')

y.download('/dataset/Справочники для проекта Рак Победим.xlsx', 'Справочники для проекта Рак Победим.xlsx')
dictionary = load_workbook('Справочники для проекта Рак Победим.xlsx')

<ipython-input-15-b3a7ef351169>:2: UserWarning: Parsing dates in %d.%m.%Y, %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  donors = pd.read_excel('Pак_победим_доноры.xlsx', parse_dates = ['Дата пожертвования', 'Date '])


## 2. Изучение данных, предобработка

### 2.1 Данные о донорах

In [ ]:
display(donors.head())
display(donors.tail())

,ID пожертвования,ID донора,Тип платежа,Плат. оператор,Способ платежа,Полная сумма,Дата пожертвования,Статус,Подписка на рассылку,Date
0,1718,ba6dd3720fda6849141f824071088a35b53e5269ff8e99...,Разовое,CloudPayments,Банковская карта,4500,2024-01-15 15:15:00,Оплачено,Нет,2024-01-15
1,1717,ba6dd3720fda6849141f824071088a35b53e5269ff8e99...,Разовое,CloudPayments,Банковская карта,4500,2024-01-15 15:10:00,Попытка пожертвовать,Нет,2024-01-15
2,1715,bc934837f1cac51334c536e54b5c574d9f64acea7d4533...,Все рекуррентные платежи,CloudPayments,Банковская карта,300,2024-01-15 12:42:00,Оплачено,Нет,2024-01-15
3,1714,41d91df30bf6a7a03a07f75a3e85304eab2913f1ecf331...,Разовое,CloudPayments,Банковская карта,750,2024-01-15 10:00:00,Оплачено,Нет,2024-01-15
4,1713,41d91df30bf6a7a03a07f75a3e85304eab2913f1ecf331...,Разовое,CloudPayments,Банковская карта,750,2024-01-15 09:55:00,Попытка пожертвовать,Нет,2024-01-15


,ID пожертвования,ID донора,Тип платежа,Плат. оператор,Способ платежа,Полная сумма,Дата пожертвования,Статус,Подписка на рассылку,Date
576,255,55a7ec19c66dfe494aa053f951c3b6d7e84610d9940dcc...,Разовое,ЮKassa,ЮMoney,500,2022-02-20 08:07:00,Попытка пожертвовать,Нет,2022-02-20
577,102,353617720091122353824e60147404469948143c427536...,Разовое,Текстовое сообщение,Дополнительные способы,210,2022-02-13 18:24:00,Оплачено,Нет,2022-02-13
578,101,353617720091122353824e60147404469948143c427536...,Разовое,Квитанции,Банковская платёжная квитанция,420,2022-02-13 18:24:00,Оплачено,Нет,2022-02-13
579,100,353617720091122353824e60147404469948143c427536...,Разовое,MIXPLAT,Платежи с помощью SMS,30,2022-02-13 18:24:00,Оплачено,Нет,2022-02-13
580,99,353617720091122353824e60147404469948143c427536...,Разовое,ЮKassa,ЮMoney,150,2022-02-13 18:24:00,Оплачено,Нет,2022-02-13


In [ ]:
# посмотрим на характеристики
describe_df(donors)

Общая информация о датасете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581 entries, 0 to 580
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ID пожертвования      581 non-null    int64         
 1   ID донора             581 non-null    object        
 2   Тип платежа           581 non-null    object        
 3   Плат. оператор        581 non-null    object        
 4   Способ платежа        581 non-null    object        
 5   Полная сумма          581 non-null    int64         
 6   Дата пожертвования    581 non-null    datetime64[ns]
 7   Статус                581 non-null    object        
 8   Подписка на рассылку  581 non-null    object        
 9   Date                  581 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 45.5+ KB


None

% пропусков по столбцам


ID пожертвования        0.0
ID донора               0.0
Тип платежа             0.0
Плат. оператор          0.0
Способ платежа          0.0
Полная сумма            0.0
Дата пожертвования      0.0
Статус                  0.0
Подписка на рассылку    0.0
Date                    0.0
dtype: float64

Количество дубликатов в датасете


0

Доля дубликатов в датасете


0.0

In [ ]:
# переименуем столбцы
donors_columns_names = {'ID пожертвования': 'id_donation',
                'ID донора':'id_donor',
                'Тип платежа': 'type_payment',
                'Плат. оператор': 'payment_operator',
                'Способ платежа': 'payment_method',
                'Полная сумма': 'amount',
                'Дата пожертвования': 'donation_date',
                'Статус': 'status',
                'Подписка на рассылку': 'mailng',
                'Date ': 'date'}
donors = donors.rename(columns = donors_columns_names)

In [ ]:
# названия листов в справочнике
print(dictionary.get_sheet_names())

['Заявитель', 'Сфера деятельности', 'Локализация', 'Стадия', 'Этап лечения', 'Помощь', 'Как узнали', 'Города']


<ipython-input-19-061a137e3c4b>:2: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  print(dictionary.get_sheet_names())


In [ ]:
donors.columns

Index(['id_donation', 'id_donor', 'type_payment', 'payment_operator',
       'payment_method', 'amount', 'donation_date', 'status', 'mailng',
       'date'],
      dtype='object')

In [ ]:
print('Количество уникальных значений')
donors.nunique()

Количество уникальных значений


id_donation         581
id_donor            229
type_payment          3
payment_operator      5
payment_method        5
amount               39
donation_date       560
status                3
mailng                1
date                269
dtype: int64

In [ ]:
# уникальные значения
for i in donors.columns:
  display(f'Количество уникальных значений в столбце {i}: {donors[i].nunique()}')
  display(donors[i].value_counts()[:10])
  print("")

'Количество уникальных значений в столбце id_donation: 581'

id_donation
1718    1
1089    1
1045    1
1044    1
1039    1
1038    1
1037    1
1036    1
1035    1
1033    1
Name: count, dtype: int64

'Количество уникальных значений в столбце id_donor: 229'

id_donor
f2518347bd38664031b51956736c2926be229ddc0e6f8dc77f863c9431926386    23
1fcda2baec2b4bf47017c102677761805271d27dbacb5763eb153811921b6e65    22
4ab4043f680b8dc8159510d77804de99df396f6e332892a81ce02a1934af2555    22
d0b6e3a433a51d7ae1a1a0a0ca72a5bda61f8e9f43c54715e77d4c431dad0fbd    21
aa1873fce203131e0b67c4aedb4d49264feb559a1dbeb4e0becd67fc0e77a1e9    18
ea4aafd0673940feeaedec88bbe93a1219ae51a2db0ef95940285b5ffd240516    18
ad014e05a33f40f69054ca3419e360873f846f88eee50c9efa18f85e8e590500    16
4391b23036696a7b6257d4c3e82b78039c54a129a5e94b3677e7bfc00ea7649b    12
e1508e75dad7c171b025a741c68fd6b3fe61fb3063b404583b519c75575e9836    12
8ba4eac592e277ee2cb7327fb58309a338fe1bb372a00a2f502a1d0659f129c7    12
Name: count, dtype: int64

'Количество уникальных значений в столбце type_payment: 3'

...

status
Оплачено                     433
Попытка пожертвовать         108
Платёж завершён с ошибкой     40
Name: count, dtype: int64

'Количество уникальных значений в столбце mailng: 1'

mailng
Нет    581
Name: count, dtype: int64

'Количество уникальных значений в столбце date: 269'

date
2023-05-14    22
2023-08-30    19
2023-04-23    11
2022-12-26    11
2023-04-27     8
2022-12-28     8
2023-11-17     8
2022-12-25     8
2024-01-15     7
2023-03-16     7
Name: count, dtype: int64

В таблице с донорами дубликаты и пропуски отсутствуют. Количество ID пожертвования соответствуют количеству строк в датасете. Замены на значения из словаря делать не нужно.


### 2.2 Данные о подопечных

In [ ]:
display(wards.head())
display(wards.tail())

,ФИО,Email,Телефон,Дата рождения,Город,Сфера деятельности,Диагноз,Этап,Помощь,Группы,Как у знали,Дата заполнения анкеты,requestid,sent,referer,Form name
0,User_01_01,Email_0001,tel_number_0001,1969-11-15,Краснодар Краснодарский край,Бухгалтер,"РМЖ, 1с стадия",Ремиссия,Информационную,Клуб “Рак Победим”,Знакомые,2021-10-21,3284862:2572982821,2021-10-21 12:02:08,http://rak-pobedim.com/anketa,Анкета участника
1,User_01_02,Email_0002,tel_number_0002,1982-08-20,г. Пушкин Санкт-Петербург,Медицина,Мальт лимфома желудка,Ещё не начала лечение,NaN,#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Интернет,2021-10-21,3284862:2575517051,2021-10-21 21:14:37,http://rak-pobedim.com/anketa,Анкета участника
2,User_01_03,Email_0003,tel_number_0003,1976-06-09,Сургут ХМАО,Пенсионер инвалид 3 гр,Рак шейки матки 3 С стадии,Лучевая терапия,Денежную. На лучевую ехать в другой город. Жил...,Клуб “Рак Победим”,Интернет,2021-10-22,3284862:2579157551,2021-10-22 16:32:42,http://rak-pobedim.com/anketa,Анкета участника
3,User_01_04,Email_0004,tel_number_0004,1960-12-19,Ростов-на-Дону,Медицина,"РМЖ, 1-ая, сделала оперативно 2 года и 9 месяц...",Принимаю гармон и прохожу раз в год полное обс...,"Уверенность и избавиться от страхов , полезную...",#ЛовлюЭмоции; Клуб “Рак Победим”; Дари Эмоции,Знакомые,2021-10-22,3284862:2579657471,2021-10-22 18:25:24,http://rak-pobedim.com/anketa,Анкета участника
4,User_01_05,Email_0005,tel_number_0005,1967-07-09,Нижний Новгород Нижегородская область,Менеджер по туризму,"РМЖ, 2 а",Гормонотерапия,NaN,#ЛовлюЭмоции; Клуб “Рак Победим”,Интернет,2021-10-24,3284862:2589075031,2021-10-24 23:49:52,http://rak-pobedim.com/anketa,Анкета участника


,ФИО,Email,Телефон,Дата рождения,Город,Сфера деятельности,Диагноз,Этап,Помощь,Группы,Как у знали,Дата заполнения анкеты,requestid,sent,referer,Form name
1563,User_01_1564,Email_1564,tel_number_1564,1987-04-03,Ростов-на-Дону,Инженер-технолог,РМЖ 1 ст.,Химиотерапия,NaN,Анонсы организации,Интернет,2021-10-12,3284862:2532127101,2021-10-12 07:50:04,http://rak-pobedim.com/anketa,Анкета участника
1564,User_01_1565,Email_1565,tel_number_1565,1979-06-02,Ростов-на-Дону,Старшая м/с,Лимфома Ходжкина,Ремиссия,NaN,Движение #ловлюэмоции; Клуб Рак победим; Анонс...,Объявление в мед.учреждении,2021-10-13,3284862:2538211561,2021-10-13 12:35:05,http://rak-pobedim.com/anketa,Анкета участника
1565,User_01_1566,Email_1566,tel_number_1566,1988-05-19,ст. Павловская Краснодарский край,Врач,РМЖ 2 В ст.,Химиотерапия,NaN,Движение #ловлюэмоции; Клуб Рак победим; Анонс...,Знакомые,2021-10-14,3284862:2542891481,2021-10-14 12:38:21,http://rak-pobedim.com/anketa,Анкета участника
1566,User_01_1567,Email_1567,tel_number_1567,1988-07-11,Батайск Ростовская область,Студент,РМЖ,Химиотерапия,NaN,Движение #ловлюэмоции; Клуб Рак победим,Объявление в мед.учреждении,2021-10-14,3284862:2545013291,2021-10-14 20:27:00,http://rak-pobedim.com/anketa,Анкета участника
1567,User_01_1568,Email_1568,tel_number_1568,1980-04-02,Санкт-Петербург,Сбербанк,РМЖ 3 ст.,Ремиссия,NaN,Клуб Рак победим,Маммофорум в Питере,2021-10-16,3284862:2550689271,2021-10-16 09:35:25,http://rak-pobedim.com/anketa,Анкета участника


In [ ]:
describe_df(wards)

Общая информация о датасете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1568 entries, 0 to 1567
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ФИО                     1568 non-null   object        
 1   Email                   1423 non-null   object        
 2   Телефон                 1568 non-null   object        
 3   Дата рождения           1493 non-null   datetime64[ns]
 4   Город                   1565 non-null   object        
 5   Сфера деятельности      1377 non-null   object        
 6   Диагноз                 1566 non-null   object        
 7   Этап                    1542 non-null   object        
 8   Помощь                  780 non-null    object        
 9   Группы                  1549 non-null   object        
 10  Как у знали             1520 non-null   object        
 11  Дата заполнения анкеты  1557 non-null   datetime64[ns]
 12  requestid           

None

% пропусков по столбцам


ФИО                        0.00
Email                      9.25
Телефон                    0.00
Дата рождения              4.78
Город                      0.19
Сфера деятельности        12.18
Диагноз                    0.13
Этап                       1.66
Помощь                    50.26
Группы                     1.21
Как у знали                3.06
Дата заполнения анкеты     0.70
requestid                  0.00
sent                       0.00
referer                    0.00
Form name                  0.06
dtype: float64

Количество дубликатов в датасете


0

Доля дубликатов в датасете


0.0

Для начала необходимо переименовать столбцы. Также в датасете есть пропущенные значения, которые невозможно заполнить подходящими данными, поэтому заменим на "Не указано".

In [ ]:
# переименуем столбцы
wards_column_names = {'ФИО': 'name',
                      'Email': 'email',
                      'Телефон': 'phone_number',
                      'Дата рождения': 'birthdate',
                      'Город': 'city',
                      'Сфера деятельности': 'fields_activity',
                      'Диагноз': 'diagnosis',
                      'Этап': 'stage',
                      'Помощь': 'help',
                      'Группы': 'group',
                      'Как у знали': 'data_source',
                      'Дата заполнения анкеты': 'date_form',
                      'Form name': 'form_name'}
wards = wards.rename(columns=wards_column_names)
wards.columns

Index(['name', 'email', 'phone_number', 'birthdate', 'city', 'fields_activity',
       'diagnosis', 'stage', 'help', 'group', 'data_source', 'date_form',
       'requestid', 'sent', 'referer', 'form_name'],
      dtype='object')

In [ ]:
# заполним пропуски
wards = wards.fillna('Не указано')

In [ ]:
# уникальные значения
for i in wards.columns:
  display(f'Количество уникальных значений в столбце {i}: {wards[i].nunique()}')
  display(wards[i].value_counts()[:10])
  print("")

'Количество уникальных значений в столбце name: 1568'

name
User_01_01      1
User_01_1054    1
User_01_1052    1
User_01_1051    1
User_01_1050    1
User_01_1049    1
User_01_1048    1
User_01_1047    1
User_01_1046    1
User_01_1045    1
Name: count, dtype: int64

'Количество уникальных значений в столбце email: 1424'

email
Не указано    145
Email_0001      1
Email_1016      1
Email_1025      1
Email_1024      1
Email_1023      1
Email_1021      1
Email_1020      1
Email_1019      1
Email_1018      1
Name: count, dtype: int64

'Количество уникальных значений в столбце phone_number: 1568'

...

sent
2021-10-21 12:02:08    1
2023-10-20 14:51:19    1
2023-10-20 08:58:15    1
2023-10-20 02:25:23    1
2023-10-19 23:51:12    1
2023-10-19 23:01:56    1
2023-10-19 21:50:14    1
2023-10-19 18:01:09    1
2023-10-18 18:47:23    1
2023-10-18 13:34:55    1
Name: count, dtype: int64

'Количество уникальных значений в столбце referer: 1'

referer
http://rak-pobedim.com/anketa    1568
Name: count, dtype: int64

'Количество уникальных значений в столбце form_name: 3'

form_name
Анкета участника      1566
Не указано               1
Предложение помощи       1
Name: count, dtype: int64

Так как данные заполняются вручную, много разночтений и опечаток, которые необходимо привести к формам, указанным в словаре

#### Города

In [ ]:
# названия листов в словаре
print(dictionary.get_sheet_names())

['Заявитель', 'Сфера деятельности', 'Локализация', 'Стадия', 'Этап лечения', 'Помощь', 'Как узнали', 'Города']


<ipython-input-28-b54cde0acbdf>:2: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  print(dictionary.get_sheet_names())


In [ ]:
cities = dictionary.get_sheet_by_name('Города')
cities = pd.DataFrame(cities.values)
cities = cities.rename(columns = cities.iloc[0]).drop(cities.index[0])
display(cities.head())
display(cities.tail())

<ipython-input-29-ffbd3d10385b>:1: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  cities = dictionary.get_sheet_by_name('Города')


,id,name,type,region,coords
1,1.0,Красный Сулин,город,Ростовская область,"[47.8815133907,40.0553169561]"
2,2.0,Сальск,город,Ростовская область,"[46.480396424,41.5374105563]"
3,3.0,Дигора,город,Республика Северная Осетия-Алания,"[43.1571481495,44.1545996296]"
4,4.0,Моздок,город,Республика Северная Осетия-Алания,"[43.7866865572,44.5958897793]"
5,5.0,Ардон,город,Республика Северная Осетия-Алания,"[43.1742767421,44.2909920696]"


,id,name,type,region,coords
1113,1113.0,Кукмор,город,Республика Татарстан,"[56.185902671,50.8967028683]"
1114,1114.0,Курчалой,город,Республика Чечня,"[43.2056205116,46.0909930997]"
1115,1115.0,Севастополь,город,Севастополь,"[44.5933976415,33.5034170869]"
1116,1116.0,Старый Крым,город,Республика Крым,"[45.0294992994,35.0888805739]"
1117,1117.0,Циолковский,город,Амурская область,"[51.7671003873,128.122306266]"


In [ ]:
# совпадения по словарю
cit = cities.name.unique()
wards[['city_2', 'city_score']] = wards.city.apply(lambda x: process.extractOne(x, cit)).apply(pd.Series)

In [ ]:
wards.head()

,name,email,phone_number,birthdate,city,fields_activity,diagnosis,stage,help,group,data_source,date_form,requestid,sent,referer,form_name,city_2,city_score
0,User_01_01,Email_0001,tel_number_0001,1969-11-15 00:00:00,Краснодар Краснодарский край,Бухгалтер,"РМЖ, 1с стадия",Ремиссия,Информационную,Клуб “Рак Победим”,Знакомые,2021-10-21 00:00:00,3284862:2572982821,2021-10-21 12:02:08,http://rak-pobedim.com/anketa,Анкета участника,Краснодар,90
1,User_01_02,Email_0002,tel_number_0002,1982-08-20 00:00:00,г. Пушкин Санкт-Петербург,Медицина,Мальт лимфома желудка,Ещё не начала лечение,Не указано,#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Интернет,2021-10-21 00:00:00,3284862:2575517051,2021-10-21 21:14:37,http://rak-pobedim.com/anketa,Анкета участника,Санкт-Петербург,90
2,User_01_03,Email_0003,tel_number_0003,1976-06-09 00:00:00,Сургут ХМАО,Пенсионер инвалид 3 гр,Рак шейки матки 3 С стадии,Лучевая терапия,Денежную. На лучевую ехать в другой город. Жил...,Клуб “Рак Победим”,Интернет,2021-10-22 00:00:00,3284862:2579157551,2021-10-22 16:32:42,http://rak-pobedim.com/anketa,Анкета участника,Сургут,90
3,User_01_04,Email_0004,tel_number_0004,1960-12-19 00:00:00,Ростов-на-Дону,Медицина,"РМЖ, 1-ая, сделала оперативно 2 года и 9 месяц...",Принимаю гармон и прохожу раз в год полное обс...,"Уверенность и избавиться от страхов , полезную...",#ЛовлюЭмоции; Клуб “Рак Победим”; Дари Эмоции,Знакомые,2021-10-22 00:00:00,3284862:2579657471,2021-10-22 18:25:24,http://rak-pobedim.com/anketa,Анкета участника,Ростов-на-Дону,100
4,User_01_05,Email_0005,tel_number_0005,1967-07-09 00:00:00,Нижний Новгород Нижегородская область,Менеджер по туризму,"РМЖ, 2 а",Гормонотерапия,Не указано,#ЛовлюЭмоции; Клуб “Рак Победим”,Интернет,2021-10-24 00:00:00,3284862:2589075031,2021-10-24 23:49:52,http://rak-pobedim.com/anketa,Анкета участника,Нижний Новгород,90


In [ ]:
# оценка меньше 90
unchange = wards[wards.city_score >= 90]
to_change = wards[wards.city_score < 90]
to_change.city.value_counts()

city
Минск Беларусь                                                5
Минск Белорусия                                               5
Не указано                                                    3
Витебск Беларусь                                              2
Королев Московская область                                    2
Алмата Казахстан                                              2
Тальменка Алтайский край                                      1
Торонто Канада                                                1
Кипения, Северный Кипр                                        1
Антрацит, ЛНР                                                 1
Душанбе Таджикистан                                           1
Брест Белорусия                                               1
Пехларн Австрия                                               1
Городок Витебская область Беларусь                            1
пгт Мостовской Краснодаский край                              1
с. Зильбачи Дагестан               

In [ ]:
# словарь с заменой российских городов
city_dict = {'Королев Московская область': 'Москва',
'Тальменка Алтайский край ': 'Барнаул',
'пгт Мостовской Краснодаский край': 'Краснодар',
'с. Зильбачи Дагестан': 'Дагестанские Огни',
'Ленинградская область': 'Санкт-Петербург',
'Орск Новоорский район с Можаровка Оренбкргская область ': 'Орск',
'п. Приютово Башкортостан': 'Белебей',
'Королëв Московская область': 'Москва',
'Тю́мень Тюменская область': 'Тюмень',
'с. Греково Степановка Чертковский район Ростовская область ': 'Ростов-на-Дону',
'Тугулым Свердловская область': 'Талица',
'Пермь Пермский край (сейчас на месяц в Сочи)': 'Пермь',
'п. Васильево Татарстан':  'Зеленодольск'   ,
'ст-ца Митякинская Тарасовский р-н Ростовская область,': 'Донецк',
'Вожега Вологодская область': 'Харовск',
'Кавалерово Примосркий край': 'Арсеньев',
'п. Трусово Астраханская область': 'Астрахань',
'Киселёвск Кемеровская область': 'Киселевск',
'р.п. Шаховская Московская область': 'Можайск',
'Казань проспект победы 139 а корпус 1 кв 95, 10 этаж': 'Казань',
'Не указано': 'Не указано',
'п. Лиман Астраханская область': 'Астрахань'}

# заменим иностранные города
to_change.city_2 = to_change.city.apply(lambda x: find_in_dict(city_dict, x, 'Зарубежье'))
to_change[['city', 'city_2', 'city_score']].head(10)

<ipython-input-33-77188ffa4b81>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_change.city_2 = to_change.city.apply(lambda x: find_in_dict(city_dict, x, 'Зарубежье'))


,city,city_2,city_score
70,"Антрацит, ЛНР",Зарубежье,68
132,Житомир Украина,Зарубежье,77
136,Гомель Белорусия,Зарубежье,68
146,Минск Белорусия,Зарубежье,68
148,Астана Кахахстан,Зарубежье,68
153,Диярбакыр Турция,Зарубежье,60
164,Королев Московская область,Москва,77
168,Минск Белорусия,Зарубежье,68
175,п. Васильево Татарстан,Зеленодольск,77
187,Вена Австрия,Зарубежье,60


In [ ]:
wards.shape

(1568, 18)

In [ ]:
wards = pd.concat([to_change, unchange])
wards.shape

(1568, 18)

In [ ]:
cities = cities.rename(columns = {'name': 'city_2'})
cities = cities.drop_duplicates(subset='city_2')
wards = wards.merge(cities[['city_2', 'region', 'coords']], how = 'left', on='city_2').fillna('Не указано')
pd.set_option('display.max_columns', False)
wards.head()

,name,email,phone_number,birthdate,city,fields_activity,diagnosis,stage,help,group,data_source,date_form,requestid,sent,referer,form_name,city_2,city_score,region,coords
0,User_01_71,Email_0071,tel_number_0071,1980-10-05 00:00:00,"Антрацит, ЛНР",Педагог,"Рак печени, 4 ст, эпителиоидная гемангоэндотел...","Химиотерапия, 5 курс",Консультацию по моему диагнощу,#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2021-12-10 00:00:00,3284862:2805026861,2021-12-10 00:40:19,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано
1,User_01_133,Email_0133,tel_number_0133,1982-04-03 00:00:00,Житомир Украина,Предприниматель,"РМЖ, трипл с мутацией, стадия 4",Химиотерапия,"Опыт лечения, Советы для облегчения побочен, С...",#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2022-01-06 00:00:00,3284862:2918786041,2022-01-06 23:32:25,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,77,Не указано,Не указано
2,User_01_137,Email_0137,tel_number_0137,1978-11-06 00:00:00,Гомель Белорусия,Бухгалтер,РМЖ 4 стадия,"Химиотераия, операция.",Не указано,#ЛовлюЭмоции; Анонсы организации; Дари Эмоции,Интернет,2022-01-07 00:00:00,3284862:2918926121,2022-01-07 00:51:01,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано
3,User_01_147,Email_0147,tel_number_0147,1976-05-28 00:00:00,Минск Белорусия,Тренер,РМЖ 3 стадии,Химиотерапия,Информацию по лечению,Клуб “Рак Победим”; Анонсы организации,Интернет,2022-01-08 00:00:00,3284862:2921812661,2022-01-08 01:00:20,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано
4,User_01_149,Email_0149,tel_number_0149,1989-03-05 00:00:00,Астана Кахахстан,Временно не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",Таргетная терапия,Не указано,#ЛовлюЭмоции; Клуб “Рак Победим”; Дари Эмоции,Интернет,2021-01-08 00:00:00,3284862:2922682461,2022-01-08 12:14:36,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано


In [ ]:
wards.shape

(1568, 20)

In [ ]:
print(dictionary.get_sheet_names())

['Заявитель', 'Сфера деятельности', 'Локализация', 'Стадия', 'Этап лечения', 'Помощь', 'Как узнали', 'Города']


<ipython-input-38-67e1b9789b9d>:1: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  print(dictionary.get_sheet_names())


#### Сфера деятельности

In [ ]:
fields_activity = dictionary.get_sheet_by_name('Сфера деятельности')
fields_activity = pd.DataFrame(fields_activity.values)
fields_activity = fields_activity.rename(columns=fields_activity.iloc[0]).drop(fields_activity.index[0])
fields_activity.head(10)

<ipython-input-39-abb24ff783ea>:1: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  fields_activity = dictionary.get_sheet_by_name('Сфера деятельности')


,Сфера деятельности пациента
1,"IT, компьютеры, интернет"
2,"Администрация, руководство среднего звена"
3,"Бухгалтерия, аудит"
4,"Гостинично-ресторанный бизнес, туризм"
5,Государственная и муниципальная служба
6,"Дизайн, творчество"
7,Инженерные специальности
8,"Красота, фитнес, спорт"
9,"Культура, музыка, шоу-бизнес"
10,"Логистика, склад, ВЭД"


In [ ]:
wards.fields_activity.value_counts()[:10]

fields_activity
Не указано     191
Домохозяйка     82
Бухгалтер       71
Пенсионер       69
Не работаю      31
Учитель         29
Экономист       23
Врач            23
Продавец        23
Менеджер        19
Name: count, dtype: int64

In [ ]:
activities = fields_activity['Сфера деятельности пациента'].to_list()
activity_df = wards.fields_activity.str.lower().to_frame().copy()

In [ ]:
activity_df[['activity', 'activity_score']] = activity_df.fields_activity.apply(lambda x: process.extractOne(x, activities)).apply(pd.Series)

In [ ]:
activity_df[activity_df.activity_score < 90].head(10)

,fields_activity,activity,activity_score
0,педагог,"Администрация, руководство среднего звена",51
3,тренер,"IT, компьютеры, интернет",60
5,домохозяйка,"Сельское хозяйство, агробизнес, животные",50
7,зав. хозяйством,"Сельское хозяйство, агробизнес, животные",63
8,"риэлтор,инвалидность 1ягруппа",Недвижимость,52
9,переводчик,"Продажи, закупки",46
10,домохозяйка,"Сельское хозяйство, агробизнес, животные",50
11,госслужащий специалист по земельным и имуществ...,Государственная и муниципальная служба,86
13,не указано,"Образование, наука",54
14,домохозяйка,"Сельское хозяйство, агробизнес, животные",50


Много не совпадений, обработаю вручную

In [ ]:
activity_dict = activity_df.groupby('activity')['fields_activity'].unique().to_dict()
for k, v in activity_dict.items():
  activity_dict[k] = v.tolist()
#activity_dict

In [ ]:
activity_dict  = {'IT, компьютеры, интернет': ['it', 'ит ', 'веб', 'web', 'програм', 'аналитик'],
 'Администрация, руководство среднего звена': ['инспект', 'администр', 'начальник'],
 'Бухгалтерия, аудит': ['бухгалтер', 'экономи', 'галтер', 'бух', 'учет', 'ревиз'],
 'Гостинично-ресторанный бизнес, туризм': ['тури', 'гид', 'гостин', 'пекар', 'кофе', 'кафе', 'ресторан', 'официан',
  'общепит',  'кулинар', 'повар', 'кондит', 'пище', 'питани', 'десерт', 'пряни', 'сыровар'],
 'Государственная и муниципальная служба': ['служа', 'воен', 'министе', 'гос', 'служб', 'полиц', 'чинов', 'войск', 'воин'],
 'Дизайн, творчество': ['дизайн', 'видео', 'design', 'графи', 'украш', 'иллюстр', 'худож', 'вязан', 'организ', 'творч', 'вяжу', 'фото'],
 'Инженерные специальности': ['конструкт', 'инженер', 'проектир'],
 'Красота, фитнес, спорт': ['красот', 'тренер', 'космет', 'шугар', 'массаж', 'макия', 'стил', 'визаж', 'ресниц', 'волос', 'бьюти', 'перманент', 'маник', 'парик', 'ногт', 'педикю', 'мейк', 'брови',
  'спорт', 'тренаж', 'йог', 'депил'],
 'Культура, музыка, шоу-бизнес': ['культур',  'певч', 'пою', 'мод', 'блог', 'развле', 'ивент', 'режиссер', 'музыка'],
 'Логистика, склад, ВЭД': ['вэд', 'внешнеэк', 'логист', 'клад'],
 'Маркетинг, реклама, PR': ['маркет', 'pr', 'реклам'],
 'Медицина, фармацевтика': ['медиц', 'медсестра', 'здравоох', 'терап',
  'фельдшер', 'акуш', 'врач', 'доктор', 'фармац', 'подолог', 'стоматол', 'педиат', 'иатр',
  'мед', 'санитар', 'провизор', 'м/с'],
 'На пенсии': ['пенси', 'бабуш'],
 'Не работаю': ['неработ', 'безработ', 'не работ', 'многодет', 'декрет', 'отпуск', 'без работ',
  'дома', 'домо', 'инвалид', 'безробот', 'д/х'],
 'Недвижимость': ['риэлт', 'недвиж', 'оценщик'],
 'Образование, наука': ['образов', 'школ', 'логоп', 'дефект',
  'педагог', 'учит', 'воспит', 'методи', 'детск', 'детьми',
  'библиотек', 'препод', 'репетит', 'обучен'],
 'Охрана, безопасность': ['охранн', 'охрана', 'безопас'],
 'Предприниматель, самозанятый': ['предприн', 'avon', 'сетев', 'фриланс', 'оконный бизнес', 'индив', 'самоз', 'ип'],
 'Продажи, закупки': ['продаж', 'закуп','представит', 'снабжен'],
 'Рабочие специальности, производство': ['оператор', 'дезинфек', 'газ', 'техник',  'механик', 'столяр', 'лабора',
  'энергет', 'машиностроит', 'промыш', 'укладчик', 'ремонт', 'производство', 'кузнец', 'техни', 'отделка', 'геммолог', 'технол'],
 'Розничная торговля': ['торгов', 'розни','опто', 'таворовед', 'кассир', 'товаровед', 'магазин', 'продове', 'продав'],
 'СМИ, издательство, полиграфия': ['филолог', 'текст', 'лингв', 'писат', 'полиграфия', 'журнал'],
 'Секретариат, делопроизводство, АХО': ['1с', 'менедж',  'архив', 'секретар', 'офис', 'делопроиз', 'хозяйствен'],
 'Сельское хозяйство, агробизнес, животные': ['ферм', 'хозяйство', 'сельск', 'животн', 'ветерин', 'зоо', 'грум', 'птиц', 'биолог', 'эколог', 'заводчик', 'растиниевод', 'садовод'],
 'Страхование': ['страхов'],
 'Строительство, архитектура, ЖКХ': ['строител',  'жкх', 'архит'],
 'Сфера обслуживания': ['переводчик', 'консульт', 'консалтинг', 'водит', 'коуч',
  'швея', 'швей', 'крой', 'ателье', 'обслуж', 'услуг', 'чист', 'колл',
  'сантех', 'психо', 'бизнес тренер', 'соц', 'флорист', 'уборщ'],
 'Телекоммуникации и связь': ['диспетч', 'коммуника', 'телеком', 'связ', 'телефон', 'коммуналь'],
 'Топ-менеджмент, руководство высшего звена': ['управленец', 'руковод', 'владе', 'директ'],
 'Транспорт, автобизнес': ['азс', 'аэропорт', 'машинист', 'жд', 'дорог', 'дорож', 'транспорт', 'авто', 'кондуктор', 'авиа'],
 'Управление персоналом': ['кадр', 'hr', 'рекрут'],
 'Учащийся, студент': ['студент', 'учусь', 'учащ'],
 'Финансы, банки': ['фин', 'банк'],
 'Юриспруденция': ['юри', 'адвок']}

In [ ]:
wards['activity'] = wards.fields_activity.apply(lambda x: find_substring(x, activity_dict, 'Другое'))

In [ ]:
wards.head()

,name,email,phone_number,birthdate,city,fields_activity,diagnosis,stage,help,group,data_source,date_form,requestid,sent,referer,form_name,city_2,city_score,region,coords,activity
0,User_01_71,Email_0071,tel_number_0071,1980-10-05 00:00:00,"Антрацит, ЛНР",Педагог,"Рак печени, 4 ст, эпителиоидная гемангоэндотел...","Химиотерапия, 5 курс",Консультацию по моему диагнощу,#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2021-12-10 00:00:00,3284862:2805026861,2021-12-10 00:40:19,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Образование, наука"
1,User_01_133,Email_0133,tel_number_0133,1982-04-03 00:00:00,Житомир Украина,Предприниматель,"РМЖ, трипл с мутацией, стадия 4",Химиотерапия,"Опыт лечения, Советы для облегчения побочен, С...",#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2022-01-06 00:00:00,3284862:2918786041,2022-01-06 23:32:25,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,77,Не указано,Не указано,"Предприниматель, самозанятый"
2,User_01_137,Email_0137,tel_number_0137,1978-11-06 00:00:00,Гомель Белорусия,Бухгалтер,РМЖ 4 стадия,"Химиотераия, операция.",Не указано,#ЛовлюЭмоции; Анонсы организации; Дари Эмоции,Интернет,2022-01-07 00:00:00,3284862:2918926121,2022-01-07 00:51:01,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Бухгалтерия, аудит"
3,User_01_147,Email_0147,tel_number_0147,1976-05-28 00:00:00,Минск Белорусия,Тренер,РМЖ 3 стадии,Химиотерапия,Информацию по лечению,Клуб “Рак Победим”; Анонсы организации,Интернет,2022-01-08 00:00:00,3284862:2921812661,2022-01-08 01:00:20,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Красота, фитнес, спорт"
4,User_01_149,Email_0149,tel_number_0149,1989-03-05 00:00:00,Астана Кахахстан,Временно не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",Таргетная терапия,Не указано,#ЛовлюЭмоции; Клуб “Рак Победим”; Дари Эмоции,Интернет,2021-01-08 00:00:00,3284862:2922682461,2022-01-08 12:14:36,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,Не работаю


In [ ]:
wards.shape

(1568, 21)

In [ ]:
print(dictionary.get_sheet_names())

['Заявитель', 'Сфера деятельности', 'Локализация', 'Стадия', 'Этап лечения', 'Помощь', 'Как узнали', 'Города']


<ipython-input-49-67e1b9789b9d>:1: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  print(dictionary.get_sheet_names())


### Локализация

In [ ]:
local = dictionary.get_sheet_by_name('Локализация')
local = pd.DataFrame(local.values)
local = local.rename(columns=local.iloc[0]).drop(local.index[0])
local.head(10)

<ipython-input-50-e25bbb731c41>:1: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  local = dictionary.get_sheet_by_name('Локализация')


,Локализация
1,Кожа (кроме меланомы)
2,Молочная железа
3,Кишечник
4,Предстательная железа
5,Тело матки
6,Легкое
7,Трахея
8,Бронхи
9,Анус
10,Почка


In [ ]:
wards.head()

,name,email,phone_number,birthdate,city,fields_activity,diagnosis,stage,help,group,data_source,date_form,requestid,sent,referer,form_name,city_2,city_score,region,coords,activity
0,User_01_71,Email_0071,tel_number_0071,1980-10-05 00:00:00,"Антрацит, ЛНР",Педагог,"Рак печени, 4 ст, эпителиоидная гемангоэндотел...","Химиотерапия, 5 курс",Консультацию по моему диагнощу,#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2021-12-10 00:00:00,3284862:2805026861,2021-12-10 00:40:19,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Образование, наука"
1,User_01_133,Email_0133,tel_number_0133,1982-04-03 00:00:00,Житомир Украина,Предприниматель,"РМЖ, трипл с мутацией, стадия 4",Химиотерапия,"Опыт лечения, Советы для облегчения побочен, С...",#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2022-01-06 00:00:00,3284862:2918786041,2022-01-06 23:32:25,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,77,Не указано,Не указано,"Предприниматель, самозанятый"
2,User_01_137,Email_0137,tel_number_0137,1978-11-06 00:00:00,Гомель Белорусия,Бухгалтер,РМЖ 4 стадия,"Химиотераия, операция.",Не указано,#ЛовлюЭмоции; Анонсы организации; Дари Эмоции,Интернет,2022-01-07 00:00:00,3284862:2918926121,2022-01-07 00:51:01,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Бухгалтерия, аудит"
3,User_01_147,Email_0147,tel_number_0147,1976-05-28 00:00:00,Минск Белорусия,Тренер,РМЖ 3 стадии,Химиотерапия,Информацию по лечению,Клуб “Рак Победим”; Анонсы организации,Интернет,2022-01-08 00:00:00,3284862:2921812661,2022-01-08 01:00:20,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Красота, фитнес, спорт"
4,User_01_149,Email_0149,tel_number_0149,1989-03-05 00:00:00,Астана Кахахстан,Временно не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",Таргетная терапия,Не указано,#ЛовлюЭмоции; Клуб “Рак Победим”; Дари Эмоции,Интернет,2021-01-08 00:00:00,3284862:2922682461,2022-01-08 12:14:36,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,Не работаю


In [ ]:
wards.diagnosis.value_counts()[:10]

diagnosis
РМЖ 2 стадия     109
РМЖ               95
РМЖ 2 ст          61
РМЖ 3 стадия      43
РМЖ 2ст           41
РМЖ 2 ст.         36
РМЖ 1 стадия      25
РМЖ, 2 стадия     25
РМЖ 4 стадия      22
РМЖ 3 ст          19
Name: count, dtype: int64

В справочнике и столбце с диагнозом значения сильно различаются. Предобработаю текст, а также создам список соответствий, используя справочник МКБ: https://rosoncoweb.ru/standarts/RUSSCO/2021/2021-mkb10.pdf

In [ ]:
y.download('/dataset/localization.csv', 'localization.csv')
localization = pd.read_csv('localization.csv', sep=',')
localization.head()

,key,value
0,Кожа (кроме меланомы),потовых желез
1,Молочная железа,молочной железы
2,Молочная железа,молочных желез
3,Молочная железа,РМЖ
4,Молочная железа,соска


In [ ]:
# список из значений
local_val = localization.value.to_list()

# предобработаю текст
values_to_check = ['стаз', 'mts', 'мтс']
full_values = ['рак', 'опухол']
pattern_1 = re.compile(rf'\b(?:{"|".join(map(re.escape, full_values))})[а-я]*\b\s*', re.IGNORECASE)
pattern_2 = re.compile(r'\b\w*(?:' + '|'.join(map(re.escape, values_to_check)) + r')\w*\b.*?(?:\s+\w+){0,3}\s*', re.IGNORECASE)
wards['loc_name'] = wards['diagnosis'].apply(lambda x: re.sub(pattern_1, '', x)).apply(lambda x: re.sub(pattern_2, '', x))
wards['loc_name'] = wards['loc_name'].apply(lambda x: x.rstrip(" "))

# сравним строки
wards[['local', 'local_score']] = wards['diagnosis'].apply(lambda x: process.extractOne(x, local_val)).apply(pd.Series)
wards.head()

,name,email,phone_number,birthdate,city,fields_activity,diagnosis,stage,help,group,data_source,date_form,requestid,sent,referer,form_name,city_2,city_score,region,coords,activity,loc_name,local,local_score
0,User_01_71,Email_0071,tel_number_0071,1980-10-05 00:00:00,"Антрацит, ЛНР",Педагог,"Рак печени, 4 ст, эпителиоидная гемангоэндотел...","Химиотерапия, 5 курс",Консультацию по моему диагнощу,#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2021-12-10 00:00:00,3284862:2805026861,2021-12-10 00:40:19,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Образование, наука","печени, 4 ст, эпителиоидная гемангоэндотелиома",Лимфома,64
1,User_01_133,Email_0133,tel_number_0133,1982-04-03 00:00:00,Житомир Украина,Предприниматель,"РМЖ, трипл с мутацией, стадия 4",Химиотерапия,"Опыт лечения, Советы для облегчения побочен, С...",#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2022-01-06 00:00:00,3284862:2918786041,2022-01-06 23:32:25,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,77,Не указано,Не указано,"Предприниматель, самозанятый","РМЖ, трипл с мутацией, стадия 4",С 97,86
2,User_01_137,Email_0137,tel_number_0137,1978-11-06 00:00:00,Гомель Белорусия,Бухгалтер,РМЖ 4 стадия,"Химиотераия, операция.",Не указано,#ЛовлюЭмоции; Анонсы организации; Дари Эмоции,Интернет,2022-01-07 00:00:00,3284862:2918926121,2022-01-07 00:51:01,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Бухгалтерия, аудит",РМЖ 4 стадия,РМЖ,90
3,User_01_147,Email_0147,tel_number_0147,1976-05-28 00:00:00,Минск Белорусия,Тренер,РМЖ 3 стадии,Химиотерапия,Информацию по лечению,Клуб “Рак Победим”; Анонсы организации,Интернет,2022-01-08 00:00:00,3284862:2921812661,2022-01-08 01:00:20,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Красота, фитнес, спорт",РМЖ 3 стадии,РМЖ,90
4,User_01_149,Email_0149,tel_number_0149,1989-03-05 00:00:00,Астана Кахахстан,Временно не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",Таргетная терапия,Не указано,#ЛовлюЭмоции; Клуб “Рак Победим”; Дари Эмоции,Интернет,2021-01-08 00:00:00,3284862:2922682461,2022-01-08 12:14:36,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,Не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",опухоль кости,86


In [ ]:
wards['localization'] = wards.merge(localization, how = 'left', left_on = 'local', right_on='value')['key']
wards.head()

,name,email,phone_number,birthdate,city,fields_activity,diagnosis,stage,help,group,data_source,date_form,requestid,sent,referer,form_name,city_2,city_score,region,coords,activity,loc_name,local,local_score,localization
0,User_01_71,Email_0071,tel_number_0071,1980-10-05 00:00:00,"Антрацит, ЛНР",Педагог,"Рак печени, 4 ст, эпителиоидная гемангоэндотел...","Химиотерапия, 5 курс",Консультацию по моему диагнощу,#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2021-12-10 00:00:00,3284862:2805026861,2021-12-10 00:40:19,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Образование, наука","печени, 4 ст, эпителиоидная гемангоэндотелиома",Лимфома,64,Злокачественные лимфомы
1,User_01_133,Email_0133,tel_number_0133,1982-04-03 00:00:00,Житомир Украина,Предприниматель,"РМЖ, трипл с мутацией, стадия 4",Химиотерапия,"Опыт лечения, Советы для облегчения побочен, С...",#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2022-01-06 00:00:00,3284862:2918786041,2022-01-06 23:32:25,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,77,Не указано,Не указано,"Предприниматель, самозанятый","РМЖ, трипл с мутацией, стадия 4",С 97,86,Множественная локализация
2,User_01_137,Email_0137,tel_number_0137,1978-11-06 00:00:00,Гомель Белорусия,Бухгалтер,РМЖ 4 стадия,"Химиотераия, операция.",Не указано,#ЛовлюЭмоции; Анонсы организации; Дари Эмоции,Интернет,2022-01-07 00:00:00,3284862:2918926121,2022-01-07 00:51:01,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Бухгалтерия, аудит",РМЖ 4 стадия,РМЖ,90,Молочная железа
3,User_01_147,Email_0147,tel_number_0147,1976-05-28 00:00:00,Минск Белорусия,Тренер,РМЖ 3 стадии,Химиотерапия,Информацию по лечению,Клуб “Рак Победим”; Анонсы организации,Интернет,2022-01-08 00:00:00,3284862:2921812661,2022-01-08 01:00:20,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Красота, фитнес, спорт",РМЖ 3 стадии,РМЖ,90,Молочная железа
4,User_01_149,Email_0149,tel_number_0149,1989-03-05 00:00:00,Астана Кахахстан,Временно не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",Таргетная терапия,Не указано,#ЛовлюЭмоции; Клуб “Рак Победим”; Дари Эмоции,Интернет,2021-01-08 00:00:00,3284862:2922682461,2022-01-08 12:14:36,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,Не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",опухоль кости,86,Кости и суставные хрящи


In [ ]:
wards.merge(localization, how = 'left', left_on = 'local', right_on='value')['key']

0         Злокачественные лимфомы
1       Множественная локализация
2                 Молочная железа
3                 Молочная железа
4         Кости и суставные хрящи
                  ...            
1563              Молочная железа
1564      Злокачественные лимфомы
1565              Молочная железа
1566              Молочная железа
1567              Молочная железа
Name: key, Length: 1568, dtype: object

In [ ]:
wards.head()

,name,email,phone_number,birthdate,city,fields_activity,diagnosis,stage,help,group,data_source,date_form,requestid,sent,referer,form_name,city_2,city_score,region,coords,activity,loc_name,local,local_score,localization
0,User_01_71,Email_0071,tel_number_0071,1980-10-05 00:00:00,"Антрацит, ЛНР",Педагог,"Рак печени, 4 ст, эпителиоидная гемангоэндотел...","Химиотерапия, 5 курс",Консультацию по моему диагнощу,#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2021-12-10 00:00:00,3284862:2805026861,2021-12-10 00:40:19,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Образование, наука","печени, 4 ст, эпителиоидная гемангоэндотелиома",Лимфома,64,Злокачественные лимфомы
1,User_01_133,Email_0133,tel_number_0133,1982-04-03 00:00:00,Житомир Украина,Предприниматель,"РМЖ, трипл с мутацией, стадия 4",Химиотерапия,"Опыт лечения, Советы для облегчения побочен, С...",#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2022-01-06 00:00:00,3284862:2918786041,2022-01-06 23:32:25,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,77,Не указано,Не указано,"Предприниматель, самозанятый","РМЖ, трипл с мутацией, стадия 4",С 97,86,Множественная локализация
2,User_01_137,Email_0137,tel_number_0137,1978-11-06 00:00:00,Гомель Белорусия,Бухгалтер,РМЖ 4 стадия,"Химиотераия, операция.",Не указано,#ЛовлюЭмоции; Анонсы организации; Дари Эмоции,Интернет,2022-01-07 00:00:00,3284862:2918926121,2022-01-07 00:51:01,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Бухгалтерия, аудит",РМЖ 4 стадия,РМЖ,90,Молочная железа
3,User_01_147,Email_0147,tel_number_0147,1976-05-28 00:00:00,Минск Белорусия,Тренер,РМЖ 3 стадии,Химиотерапия,Информацию по лечению,Клуб “Рак Победим”; Анонсы организации,Интернет,2022-01-08 00:00:00,3284862:2921812661,2022-01-08 01:00:20,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Красота, фитнес, спорт",РМЖ 3 стадии,РМЖ,90,Молочная железа
4,User_01_149,Email_0149,tel_number_0149,1989-03-05 00:00:00,Астана Кахахстан,Временно не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",Таргетная терапия,Не указано,#ЛовлюЭмоции; Клуб “Рак Победим”; Дари Эмоции,Интернет,2021-01-08 00:00:00,3284862:2922682461,2022-01-08 12:14:36,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,Не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",опухоль кости,86,Кости и суставные хрящи


In [ ]:
wards.localization.value_counts()

localization
Молочная железа                                      943
Шейка матки                                          112
Кишечник                                              76
Яичник                                                76
Злокачественные лимфомы                               54
Желудок                                               43
Легкое                                                38
Щитовидная железа                                     24
Кости и суставные хрящи                               23
Тело матки                                            23
Поджелудочная железа                                  21
Множественная локализация                             17
Соединительная и другие мягкие ткани                  14
Почка                                                 14
Предстательная железа                                 12
Без уточнения локализации                             12
Меланома кожи                                         11
Лейкемии          

In [ ]:
wards.shape

(1568, 25)

In [ ]:
local = dictionary.get_sheet_by_name('Локализация')
local = pd.DataFrame(local.values)
local = local.rename(columns=local.iloc[0]).drop(local.index[0])
local.head(10)

<ipython-input-60-e25bbb731c41>:1: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  local = dictionary.get_sheet_by_name('Локализация')


,Локализация
1,Кожа (кроме меланомы)
2,Молочная железа
3,Кишечник
4,Предстательная железа
5,Тело матки
6,Легкое
7,Трахея
8,Бронхи
9,Анус
10,Почка


### Стадия

In [ ]:
phases = dictionary.get_sheet_by_name('Стадия')
phases = pd.DataFrame(phases.values)
phases = phases.rename(columns=phases.iloc[0]).drop(phases.index[0])
phases.head(7)

<ipython-input-61-70060b812c26>:1: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  phases = dictionary.get_sheet_by_name('Стадия')


,Стадия
1,0 (in situ)
2,1.0
3,2.0
4,3.0
5,4.0
6,Не установлена


Стадия находится в столбце `diagnosis`, из которого ее необходимо выделить. Если посмотреть на данные, то в значениях встречаются рисмкие цифры и стадии рака по системе TNM

In [ ]:
# Создадим регулярное выражение для поиска римских цифр от I до IV. Начинаем с 4, чтобы ее вообще можно было найти. Ищем сначала 4, а потом 1, которая повторяется от 1 до 3 раз
pattern_1 = re.compile(r'(IV|I{1,3})')

# Функция для замены римских цифр на арабские
def replace_roman(match):
    return str(roman.fromRoman(match.group(0)))

pattern_2 = re.compile(r'(\d)\s*(?:стадия|ст)\b|\b(?:стадия|ст)\s*(\d)|\b(\d)(\D+|$)|(?:t|т|ршм)\s*(\d)', re.IGNORECASE)

# Извлекаем цифры из каждой строки столбца
wards['stage_ill'] = wards['loc_name'].apply(lambda x: re.sub(pattern_1, replace_roman, x))\
  .apply(lambda x: re.search(pattern_2, x) \
  .group(1) or re.search(pattern_2, x).group(2) or re.search(pattern_2, x).group(3) or re.search(pattern_2, x).group(5) if re.search(pattern_2, x) else 'Не указана')

In [ ]:
wards.stage_ill.value_counts()

stage_ill
2             557
3             321
Не указана    309
4             195
1             181
0               3
5               1
8               1
Name: count, dtype: int64

In [ ]:
# посмотрев на значения, заменим вручную
wards.loc[wards.stage_ill == '0', 'stage_ill'] = 'Не указана'
wards.loc[wards.stage_ill == '5', 'stage_ill'] = 'Не указана'
wards.loc[wards.stage_ill == '8', 'stage_ill'] = 'Не указана'

### Этап лечения

In [ ]:
stage_treatment = dictionary.get_sheet_by_name('Этап лечения')
stage_treatment = pd.DataFrame(stage_treatment.values)
stage_treatment = stage_treatment.rename(columns=stage_treatment.iloc[0]).drop(stage_treatment.index[0])
stage_treatment.value_counts()

<ipython-input-65-aea7eb5e9366>:1: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  stage_treatment = dictionary.get_sheet_by_name('Этап лечения')


Этап лечения         
Гормональная терапия     1
Иммунотерапия            1
Лучевая терапия          1
Постановка диагноза      1
Реабилитация             1
Ремиссия                 1
Рецидив                  1
Таргетная терапия        1
Химиотерапия             1
Хирургическое лечение    1
Name: count, dtype: int64

In [ ]:
wards.stage.value_counts()[:10]

stage
Ремиссия             321
Химиотерапия         288
Гормонотерапия        41
Таргетная терапия     34
Не указано            26
Химия                 26
Лучевая терапия       24
ремиссия              22
Операция              16
химиотерапия          16
Name: count, dtype: int64

In [ ]:
stage_tr = stage_treatment['Этап лечения'].to_list()
wards[['stage_treatment', 'stage_score']] = wards.stage.apply(lambda x: process.extractOne(str(x), stage_tr)).apply(pd.Series)
wards.head()

,name,email,phone_number,birthdate,city,fields_activity,diagnosis,stage,help,group,data_source,date_form,requestid,sent,referer,form_name,city_2,city_score,region,coords,activity,loc_name,local,local_score,localization,stage_ill,stage_treatment,stage_score
0,User_01_71,Email_0071,tel_number_0071,1980-10-05 00:00:00,"Антрацит, ЛНР",Педагог,"Рак печени, 4 ст, эпителиоидная гемангоэндотел...","Химиотерапия, 5 курс",Консультацию по моему диагнощу,#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2021-12-10 00:00:00,3284862:2805026861,2021-12-10 00:40:19,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Образование, наука","печени, 4 ст, эпителиоидная гемангоэндотелиома",Лимфома,64,Злокачественные лимфомы,4,Химиотерапия,90
1,User_01_133,Email_0133,tel_number_0133,1982-04-03 00:00:00,Житомир Украина,Предприниматель,"РМЖ, трипл с мутацией, стадия 4",Химиотерапия,"Опыт лечения, Советы для облегчения побочен, С...",#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2022-01-06 00:00:00,3284862:2918786041,2022-01-06 23:32:25,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,77,Не указано,Не указано,"Предприниматель, самозанятый","РМЖ, трипл с мутацией, стадия 4",С 97,86,Множественная локализация,4,Химиотерапия,100
2,User_01_137,Email_0137,tel_number_0137,1978-11-06 00:00:00,Гомель Белорусия,Бухгалтер,РМЖ 4 стадия,"Химиотераия, операция.",Не указано,#ЛовлюЭмоции; Анонсы организации; Дари Эмоции,Интернет,2022-01-07 00:00:00,3284862:2918926121,2022-01-07 00:51:01,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Бухгалтерия, аудит",РМЖ 4 стадия,РМЖ,90,Молочная железа,4,Химиотерапия,83
3,User_01_147,Email_0147,tel_number_0147,1976-05-28 00:00:00,Минск Белорусия,Тренер,РМЖ 3 стадии,Химиотерапия,Информацию по лечению,Клуб “Рак Победим”; Анонсы организации,Интернет,2022-01-08 00:00:00,3284862:2921812661,2022-01-08 01:00:20,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Красота, фитнес, спорт",РМЖ 3 стадии,РМЖ,90,Молочная железа,3,Химиотерапия,100
4,User_01_149,Email_0149,tel_number_0149,1989-03-05 00:00:00,Астана Кахахстан,Временно не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",Таргетная терапия,Не указано,#ЛовлюЭмоции; Клуб “Рак Победим”; Дари Эмоции,Интернет,2021-01-08 00:00:00,3284862:2922682461,2022-01-08 12:14:36,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,Не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",опухоль кости,86,Кости и суставные хрящи,1,Таргетная терапия,100


In [ ]:
stage_dict = wards.groupby('stage_treatment')['stage'].unique().to_dict()
for k, v in stage_dict.items():
  stage_dict[k] = v.tolist()
stage_dict

{'Гормональная терапия': ['Химии гормональные',
  'Гормонотерапия',
  'Начало',
  'Начальная',
  'Только узнала',
  'Не начато',
  'Восстановление после резекции правой молочной железы, радикально с региональной лимфаденэктомией',
  'Гормональнотеррапия',
  'Эндокринная терапия',
  'Начальный',
  'Контроль',
  'Не назначали',
  'Гормональная терапия',
  'Гормоны',
  'гормонотерапия',
  'Прием гормон',
  'Послеоперационный период',
  'Контрольные анализы',
  'Таргетная терапия + гормональная терапия',
  'не назначены',
  'Гармональное',
  'Прошла 8 курсов химеотерапии, операцию, впереди лучевая и гормональная терапия',
  'Ремиссия , гормональная терапия',
  'Скоро операция',
  'Антигормональная терапия после химии и операции',
  'Продолжается химия',
  'Гормональная терапия, приём ИА',
  'Гормональное лечение',
  'На контроле',
  'Лучевая и гормональная терапия',
  '22 ноября операция',
  'на гормонотерапии',
  'Выздоровела',
  'Гормоноерапия',
  'Здорова, на учете',
  'Сейчас ремиссия.

In [ ]:
dict_stage = {('Химии гормональные', 'Гормонотерапия', 'Гормональнотеррапия', 'Эндокринная терапия', 'Гормональная терапия', 'Гормоны', 'гормонотерапия', 'Прием гормон',
                          'Гармональное', 'Антигормональная терапия после химии и операции', 'Гормональная терапия, приём ИА', 'Гормональное лечение', 'гормональная терапия'
                          'на гормонотерапии', 'на гормонотерапии', 'Гормоноерапия', 'Гормональная терапия, резкое ухудшение', 'Наблюдение, гормонотерапия, после радикальной мастэктомии','Гармонотерапия',
                          'Гормонотерапия, иммунотерапия', 'Эндокринотерапия', 'Лечение гормонотерапией',  '2-год, гормонотерапия', '4 этап, гормонотерапия ингибиторами ароматазы',
                          'Гормонотерапия, планируемая операция 20.09.2023', 'Гармонотерапии',  'Гормонотерапия, бисфосфонаты', 'Поддерживающая гормонотерапия',
                          'Принимаю гармон и прохожу раз в год полное обследования',  'Гормонотерпия, лучевая', 'Гормонатерапия и тагерная терапия', 'Гармоно-таргентная терапия',
                          'Ингибиторы аромотазы', 'Паллеативное лечение-бусерелин', 'лечение ( тамоксифен)', 'Гормональное лечение, после операции',
                          'Анастрозол, Золодекс 3.6,золидроновая кислота, Рибоциклиб 0.600', 'Тамоксифен',  'Таблетки мекинист,Тафинлар','Золидронку и таксифен получаю',
                          'Продолжение трастузумаба после операции и лучевой терапии', 'Золадекс и летрозол.','Адъювантная терапия тамоксифеном',
                          'Получаю антитела (ниволумаб)', 'Тамоксифен 5 лет', 'Приём лекарств', 'Таблетки тамоксифен ещё 3 года.',
                          'Таблетки', '1 этап таблетками'): 'Гормональная терапия',

               ('Имунотерапия', 'Иммуннотерапия', 'Иммунотерапия', 'иммунотерапия', 'Иммунная терапия', 'Иммунотерапия и таргетная терапия', 'имунотерапия',
                   'Иммунотерапия, ожидание операции',  '3 линия, иммунотерапия Ниволумабом',
                   'Противоопухолевая биотерапия', 'Иммунотерапия + предстоит скорое оперативное лечение (5-е по счету)', 'Иммунотерапия+химиотерапия'): 'Иммунотерапия',

               ('Лучевая терапия', 'Лучевая и гормональная терапия', 'Лучи', 'Химиолучевая терапия', 'Лучевая', 'Радио лучевая терапия',
                     'лучевая терапия', 'Направлена на лучевую терапию после радикальной мастэктамии', 'Жду лучевую тератию', 'Прошла операцию,химию и вот закончила лучевую',
                     'В предверии лучевой терапии', 'лучева терапия', 'Лучевая, таргет терапия', 'после химии, лучевая', 'После мастектомии, в ожидании химии и лучевой',
                     'Пройдены тотальное облучение головного мозга, лучи на грудь. Первая линия ХТ.', 'После операции и лучевой',
                     'Прооперирована 2 недели назад, в октябре предстоит лучевая терапия', 'После лучевай', 'метостазы; новый курс лучевой терапии; без гарантий...', 'Лучевая и химия',
                     'Заканчиваю лучевую терапию', 'После лт', 'Радио йод сделали будет повторно, нашли метастаз подмышечный говорят надо удалить',
                     'После операции, жду лучи', 'Радиойодтерапия',  'Радиотерапия', 'В ожидании лучей после химии и операции', 'Было 3 курса химиотерапии, а сейчас отправляют на лучи',
                     'Облучение', 'Прошёл облучение', 'Прохожу лечение - лучи и химию', 'Лучевая терапия (после операции)'): 'Лучевая терапия',

               ('Не указано', 'Установлен диагноз', 'Поставили диагноз', 'Жду результатов игх', 'Обследование', 'Только прошла обследование, жду игх', 'Постановка диагноза',
                         'Пока никакой', 'Онкологи отказали', 'Только поставлен диагноз', 'Обследования', 'Диагностика', 'Дообследование', 'Сдаю анализы', 'Больше года',
                         'Только узнали диагноз', 'Только диагностирован, готовлюсь к лечению', 'Обсудить',
                         'Направлена на анализ игх для уточнения диагноза и плана лечения. Сделана резекция. Сделана подкожная мастэктомия',
                         'Вчера узнала диагноз, пришел результат гистологии', 'Жду', 'Только узнали', 'Зависла на плохих анализах', 'Начало', 'Начальная', 'Только узнала',
                         'Не начато','Начальный', 'Контроль', 'Не назначали','Контрольные анализы', 'не назначены', 'На контроле', 'Консультация радиолога',
                         'На этапе обследования на кэп кт', 'несколько месяцев пыталась получить помощь в онкодиспсере на Соколова 9. От основного заболевания пошли осложнения. Онколог отправил меня решать свои проблемы самой. Но я же врач. после обследований в начале лета 2021 прошло много времени. поэтому я прошла повторные обследования в этом месяце в онкоцентре на 14 линии. Там мне могут помочь. Но, из-за болезни я не могу работать. и инвалидность оформить не могу. меня никто не лечит. Гормонотерапия, таргетная терапия ',
                         'Обследование и операция', '4я стадия', '1 этап', 'Начало пути, обследование', 'Узнала о заболевании три дня назад', 'Впервые заболела',
                         'Только получили результат биопсии','Госпитализация после нг','4я стадия','Ждём гистологию','Прогрессия в левую молочную железу', 'Еще в процессе диагностики',
                         'Прогрессирующая', 'Регрессия', 'Пока прохожу обследование и жду заключение специалистов',  'Только узнали о заболевании', 'Обследование, подготовка к операции',
                         'На этапе уточнения стадии', 'Жду назначения по кт.на кт очередь до мартапо полису омс.', 'Ждут назначения', 'На этапе прохождения обследования',
                         'Ещё не начала лечение','Пока нет лечения', 'Жду назначение лечения',
                         'Жду начало лечения', 'Обследование перед лечением', 'Начало лечения',  'Не было', 'Ищу лечение', 'начало, собираю анализы, ищу место лечения',
                         'Еще не назначено. Пересмотр стекол, блоков (ГА+ИГХ)','Еще нет лечения', 'Дообследование, выбор лечения', 'Жду трепан-биопсию и назначение лечения', 'Первичный',
                         'Врачи в поиске решений.',  'Ожидание назначения для лечения', 'Обследовпние', 'Ожидание лечения',
                         'В декабре2020 завершила все лечение. На данный момент прохожу обследование с подозрением на метастазы в легком.'): 'Постановка диагноза',

                 ('Стабилизация', 'Реконструкция', 'В июне закончила ХТ', 'Реабилитация после мастэктомии', 'Год стабилизации', 'Стабилизация+ золедроновая кислота',
                  'Восстановление после резекции правой молочной железы, радикально с региональной лимфаденэктомией', 'Послеоперационный период',  'Иммунотерапия, реабилитация',
                  'После операционный период', 'После лучевой терапии', 'П/операции', 'Стабилизация процесса после операции и лучевой терапии', 'После лучевой и укладок',
                  'восстановление после операции и облучения', 'Послеоперационный', 'Послеоперационная', 'Послеоперационное восстановление','Восстановление после химии',
                   '1,5 года после удаления и 1 год после химии', 'После химии, лучей',  'После химиии', 'Наблюдение каждые 3 мес.', 'Супрессия', 'После операции', 'После операции и лт',
                  'После операции 07.04.2022 г', 'Наблюдение после атипичной резекции', 'После операции.', 'После операции удаления образования, в больнице',
                  'Восстановление после операции', 'Только после операции', 'Год после операции, пью таблетки', 'Отдых после лучевой и химиотерапии',  'Послеоперационный,скоро химиятерапия',
                  'Постоперация','После операции, химиотерапии, лучевой', 'восстановление после химиотерапии', 'После химиотерапии и операции', 'Востановление после химиотерапии',
                  'После операции и химиотерапии под наблюдением', 'Год после операции, химиотерапии, лучевой.', 'Состояние после комплексного лечения в2019 ,впередиоперации по профилактике и рекнструкции',
                  'была операция, потом радиотерапия, сейчас наблюдение у онкогинеколога',  'После лучевой и химиотерапии', 'Полгода после лечения',
                  'Закончила лечения', 'Наблюдение',  'Только окончено активное лечение', 'Сейчас после химии наблюдение', 'Только закончила лечение', 'После лечения',
                  'Набледение после лечения', 'Окончила лечение в августе 2020', 'Динамическое наблюдение после операции', 'Закончила лечение', 'Поддерживающая терапия'): 'Реабилитация',

                 ('Ремиссия 2 года', 'Ремиссия', 'ремиссия', 'Ремиссия 1 год', 'Вылеченный! Ремиссия стойкая.', 'Реммисия', 'Ремиссия.пью тамоксифен.', 'Ремисия', 'Ремессия',
                           'Ремиссия, гормонотерапия','Ремиссии', 'Ремиссия принимаю химиопрепараты', 'Ремиссия и лечение', 'Послеоперационный период, ремиссия', 'Операция, ремиссия',
                           'Ремиссия.', 'Римисия', 'Ремисси','1 год ремиссия', 'Ремиссия, лимфостаз левой руки', 'Ремиссия, этапы реконструкции груди', 'Ремиссия 3 год', 'ремисия',
                           'Ремиссия, реконструкция груди', '4 год ремиссии', 'ремиссия, гормонотерапия', 'ресмиссия','3 года ремиссии','Ремиссия, наблюдения 1 раз в 6 мес.', 'Ремиисия',
                           'Полное ремиссия', 'Частичная ремиссия', 'Гормонотеропия. Ремиссия', 'Ремиссии.', 'ремиссии', 'Ремиссич',
                           'Ремиссия это полное излечение, это лет через 30 можно будет понять. Говорить сейчас о чем-то подобном, на мой взгляд, как кривляние какое-то.Глупо это.Виднла я девочек в &quot;ремиссии&quot;полугодичной, которые умирали. Я очень осторожно отношусь к этому слову и не применяю его. Лет через 30 поговорим об этом ;))',
                           'Ремиссия 3,5 года', 'После лечения -Ремиссия', 'Ремиссия.Рецидив?', 'Неуверенная ремиссия', 'Ремиссия пол года', 'Ремиссия с приемом препаратов',
                           'Медикаментозная ремиссия', 'Ремиссия , гормональная терапия',  'Выздоровела',  'Здорова, на учете',
                           'Сейчас ремиссия. До этого: 12 курсов хт; операция; 10 курсов поддерживающей хт; рецидив; лучевая терапия. Сейчас состояние ремиссии после лучевой терапии',
                           'Ремиссия это полное излечение, это лет через 30 можно будет понять. Говорить сейчас о чем-то подобном, на мой взгляд, как кривляние какое-то.Глупо это.Виднла я девочек в &quot;ремиссии&quot;полугодичной, которые умирали. Я очень осторожно отношусь к этому слову и не применяю его. Лет через 30 поговорим об этом ;))',
                           'Ремисия,иммунотерапия', 'Ремиссии, гормонощампмтительная терапия'): 'Ремиссия',

                  ('Рецедив','Рецидив', 'Рецидив после операции и лучевой терапии','Рецидив, лечение', 'Второй рецедив,химич', 'Рецидив через 2 года',
                          'Жду результат анализа, подозрение на рецидив', 'рецидив', 'Рецидив.ХТ.', 'Лечение , после рецидива', 'Рецидив. Лечение.',
                          'Химиятеропия рецидив', 'Циторедукция в октябре 2020, в процессе лечения', 'Рецедив, таргетное, гармонотерапия.', 'Рецидив, вторая химия.',
                          'После удаления желудка,химия, рецедив,химия', 'Рецедив mts кости',  'Была в ремиссии, случился рецедив, прошла операцию, 1 курс химитеропии, всего назначено 6',
                          'В процессе химиотерапии. Рецедив', 'Рецидив, химиотерапия', 'Рецидив тнрмж пошли метостацы в кожу химиотерапию отменили омэ', 'Химиотерапия, рецидив', 'Рецедив, химиотерапия',
                          'Операция, химиотерапия, через 3 месяца рецедив, химиотерапия+таргеты'): 'Рецидив',

                  ('Таргетная терапия', 'Таргетная терапия после операции;', 'Таргеты', 'Таргетная терапия до февраля', 'После операции, таргетная терапия',
                       'На таргетной терапии', 'Таргетная', 'Таргертная терапия', 'Гормональная+таргетная терапия+бисфосфонаты', 'Таргет', 'Таргентная терапия',
                       'таргетная терапия', 'По РЛ таргетная терапия, по РМЖ пока без терапии', 'Химия+таргет', 'Таргерт', 'Прохожу таргетную терапию', 'Таруентная терапия',
                       'Таргетная терапия + Радиотерапия', '10 курс таргетной терапии', 'Таргентная плюс гормонотерапия', 'Таргетная терапия + тамоксифен',
                       'Курс химиотаргетная терапия 5й курс', 'Таргетная терапия Олапариб', 'Таргетная терапия олапариб', 'Таргетная химия', 'Таргетная и гормонотерапия',
                       'Таргентная химия', 'Таргетная терапия + золедроновая кислота', 'Таргетная терапия. До этого (с июня 2019) химиотерапия, операция, лучевая, метостазы в легкое.. опять операция, химия.',
                       'Таргетная терапия, гормонотерапия.', 'Хт не принесла результата. На данный момент таргетная терапия',
                       'Таргет, гормоны, бифосфонаты', 'Химиотерапия, таргетная терапия', 'Гормонотерапия, таргетная терапия',
                       'Таргент', 'Таргетная терапия + гормональная терапия',  'Таргеты м гормонотерапия', 'Таргеиная терапия после 4 рецедива',
                       'Таргент. терапия. Гормонотерапия', '4 линия: КИ, таргет', 'Химиотерапия, гормонотерапия', 'Принимаю олапариб',
                       'Эндокринотерапия после операции, химиотерапии и лучевой', 'Химиотерапия таргетная в таблетках,восстановление',
                       'В марте операция, в июле закончила 6 химию, сейчас 22 таргетной назначили', 'Химиотерапия/гормонотерапия',  'Таргетное лечение',
                       'стабилизация( таргегное/ гормональное лечение', 'лечение ингибиторами ароматазы', 'Продолжаю лечение. Золедроновая и Анастрозол',
                       'Таргетное пожизненное лечение даже в ремиссии. В ремиссию пока не вошла.', 'Гормонотерапия (только закончилась таргетная)', 'Гормонотерапия таргетная терапия',
                       'Пройдена химия, двусторонняя мастоэктомия, лучевая терапия, сейчас гормонотерапия, в ближайшее время удаление яичников'): 'Таргетная терапия',

                  ('Химиотерапия, 5 курс','Химиотерапия', 'Химиотераия, операция.', 'Химиотерапия,подготовка к операции', 'Назначают химиотерапию', 'Химия', 'химиотерапия', 'Химиотҽраия',
                  'Закончила химию терапию, готовлюсь к операции','Была химиотерапия,лучевая.Сейчас пью химию в таблетках.','химиятерапия', 'Химиятеропия', 'Химиотерапии',
                  'Химиотерапия , таргетная терапия', 'Химиятерапия', '2 курса химии', '1 химия', 'Химиотерапия, операция','Начало химиотерапии',
                  'Химиотерапия операция', 'Химиотерапии под???', 'Химиотерапия, 7 курс, 2 операции на позвоночнике ( установка импланта-Т10 позвонка-грудной отдел)))',
                  'химиотерапия в таблетках (капецитабин)', '8 курсов хт прошла, ещё 8 впереди и операция', 'Между химией и лучевой', 'Химиотерапия, подготовка к операции',
                  'закончила курс химиотерапии', 'Лечение химиотерапией', 'Прогрессирование, химиотерапия', 'Масэктомия 15.09, химиотерапия', 'Химиотерапия, 3 линия',
                  'Закончила курс химиотерапии','Химии терапия', 'Терапия', 'Лучевая химиотерапия', 'Химиотерапия Капецитабин 2 курс', 'Вторая линия химиотерапии',
                  'Хитмиотерапия', 'Направление на химиотерапию', '1 химия 20.03.2022', 'Химиотеропия', 'Химиотерапия 2 линия', 'ХТ', 'Прошла химиотерапию, операцию',
                  'Химиотерапия после операции', 'Очередной курс химиотерапии', 'Хт', 'Химиотерапия таблетированная', 'за заХимиотерапии', 'Теперь уже постоянная химиятерапия',
                  'Химиотерапия, таргетная терапия', 'Начало, химия террепия.', 'химеотерапия', 'Сделаны химиотерапия и операция, жду лучевую','4 химия терапия',
                  'После химиотерапии , операции, лучевой', 'химиотерапия перед операцией', '8 химий', 'Химиолучевая', 'В июле последняя химия направляют в Москву не получилось в герцина',
                  'НХТ, мастэктомия', 'Химиотерапия после мастэктомии', 'Послеоперационная химтотерапия', 'Химия терапея', 'Химия, 6ой курс','Неоадьювантная химиотерапия',
                  'Химиотерапия (впереди)','Рецидив тнрмж пошли метостацы в кожу химиотерапию отменили омэ', 'Химитерапия', 'Сначала химия потом операция', 'Химии',
                  '3 химиотерапии, 29.03 будет 4', 'Закончила химиотерапию', 'ПРОХОЖУ ХИМИЮТЕРАПИЮ', 'Химиотерпия', 'химиотерапия после операции', 'Сейчас идёт 5 линия химиотерапии',
                  '2022_2023 хлт', 'Этап химиотерапии', '4 химиотерапия', '11 химиотерапия', 'Химеотерапия', 'Закончила химиотерапию, жду операцию', 'Закончена химиотерапия',
                  'Закончилась химиотерапии и проведена радикальная мастэктомия', 'Прошла химиотерапию, впереди гормонотерапия', 'Прогрессирование в костях,химиотерапия',
                  'Пошла две химиотерапии', 'У мамы химиотерапия', 'сделала2 химиотерапии', 'Диагностика, 1 курс химиотерапии 31.07', 'Йодтерапия',
                  'Химиотерапия 8 курсов', 'Химиотерапии, операция, возможно омпутация', 'Химиотерапия каждые 28 дней', '2 химии терапии',
                  'Прошла курс лучевой и химиотерапии, впереди еще 6 курсов химии дома надо пропить', 'Две химии', 'Химиеотерапия', 'Через неделю начинаю химию',
                  'Месяц после операции, в процессе назначения химиотерапии', 'Жду химиотерапию', 'Химиотерапия перед операцией', '4 химия',
                  'химиотҽрапия', 'После оперативного лечения. Химиотерпия', 'После химии', 'Пройдена химия, операция, лучи. Сейчас капицетабин до июня 2021 года',
                  'Химия, операция.', 'Химотерапия', 'Прошла 8 химиотерапии', 'Химиотерапия и лучи', 'Химиотерапия, симптоматическая терапия печени',
                  'Химиотеоапия',  'Паллиативная химиотерапия', '6 блоков химиотерапии; 13 поддерживающей химии капецитабин', 'Химиотерапия 6 курсов, ретуксимаб принимаю',
                  'Химиотерапия, лечусь 3 года', 'Высокодозная химиотерапия с последующей перестройкой костного мозга', 'Предстоит химиотерапия', 'После операции химиотерапия',
                  'Химиотерапия.', 'Химиотерапия мо', 'Химиотерапия начало', 'Химиотерапия первая', 'Химиотерапия-операция-лучевая', 'Химиотерпапия',  '5й год лечения (химия)',
                  'Капельницы', 'Прошла 4 химии, жду приёма врача о решении дальнейшем дальнейших действий', 'Уколы, таблетки', 'Продолжается химия',
                  'Была операция, удалена правая грудь, сейчас назначили 6 курсов химиотерапии , прошла 2 курса пока',
                  '08.01.2022 последняя хт из 16', 'Проведена операция, сейчас Химия терапия', 'Химия терапия, возможно будет операция кибер нож',
                  'Пройдено: 8 курсов химии, операция, лучевая терапия', 'Химия терапия после операций', 'Химия терапия в таблетках', 'Химия и лучевая', 'хлт',
                  'Прошла 8 курсов химеотерапии, операцию, впереди лучевая и гормональная терапия', 'метрономная терапия это ребенок 10 лет я его мама', 'НАХТ',
                  'Прошла 4 курса красной химии', '8 хт по протоколу mFolfirinox', '3курс ПХТ после операции', 'После операции, назначили химию',
                  'Сделана операция, впереди химия', 'Ожидание 2 линии высокодозной химии', 'ХТО 4 курс АС',
                  'Прохождение химии',  '3курс ПХТ после операции',  'Предстоит химия', 'Прохожу химию после секторал.операции.',
                  'Химия терапия', 'После химиотерапии', 'Осталось 2 курса по схеме ABVD'): 'Химиотерапия',

                  ('Не указано', 'В лечении', 'Лечение повторных метастаз', 'Лечение',  'активное лечение', 'Комплексное лечение',  'Мтс, лечение',  'Прохожу лечение',
                    'Лечение опухоли', 'На лечении', 'лечение', 'Лечение метастаз в костях',  'В процессе лечения', 'в процессе лечения.', 'лечение', 'Активное лечение',
                   'Обострение',  'Наблюдение , лечение', 'Прогресс заболевания мтс в л/узлах ворота печени', 'Терапия по мтс в легком', '?',
                   '5 лет ремиссии, потом МТС в кости', 0):  'Не указано',

                  ('отказ от лечения', 'Без лечения',  'Паллиативное лечение',  'Метастазы на брижейке, в химии отказали, маленький вес.', 'Отказ от химиотерапии'): 'Без лечения',

                  ('Сделали операцию, лучеваятерапия и химиотерапия', 'Операция, ХТ', 'Операция 21.06.2021',
                           'После операции ,лечение пока не назначено ,нахожусь в стационаре онкодиспансер а на Соколова 9 в Ростове на Дону', 'Хирургическое лечение',
                           'ТУР','Операционное лечение', 'Оперативное лечение', 'Лечение после операции', 'После удаления', 'Радиохирургия',
                           'Операция, таргет', 'Радикальная мастэктомия по Маддену,4 химиотерапии, 17 таргетной терапии и анастрозол',
                           'Скоро операция', '22 ноября операция',  'На днях была операция', '26.07. Ложусь в больницу, на операцию',  'Назначение на операцию',
                           'Операция предстоит на гм', 'Операция март 2022, гормонотерапия', 'Жду операцию',  'Операция после 8 курсов химии.',
                           'Оперативное вмешательство', 'После операции удаление щитовидной железы,жду гистологию.дной железы',
                           'Операция ,химия ,лучи ,сейчас таблетки и укол',  'Органосохранная операция резекция, лучевая терапия',  'Месяц назад полное удаление правой груди',
                           'Подготовка к удивлению почки', 'Трансплонтация костного мозга', 'Конизация', 'Только удалили молочную железу, дальше химия',
                           'Первая стадия операция ,на шрамы 2 раза рецидивист, по ом удаление груди и 14 лифоузлов', 'Жду опрацию',
                           'Операция,резекция хвоста ПЖ. Готовлюсь к химии.', 'После операции, назначили химию', 'Сделана операция, впереди химия', 'Операция опухоли'
                           'Предстоит удаление груди', 'Пройдено две операция ,радиойодолечение впереди', 'Ожидаю операции', 'Подготовка к операции', 'подготовка к операции',
                           'Накануне операции', 'Предоперационный', 'Второе Операционное вмешательство',  'Планируется операция', 'Сублобарная атипичная резекция верхней доли легкого',
                           'Операция, 6 хт и 18 ТТ, сейчас на контроле', 'Проведена операция в августе 2018 года', 'Операция', 'Предстоит операция 22.02',
                           'Операция по удалению опухоли', 'Операция по удалению', 'Операция,резекция хвоста ПЖ. Готовлюсь к химии.', 'Предстоит удаление груди', 'Подготовка к операции'): 'Хирургическое лечение'}

In [ ]:
wards['stage_treatment'] = wards.stage.apply(lambda x: find_in_dict(dict_stage, x, 'Не указано'))
wards.head()

,name,email,phone_number,birthdate,city,fields_activity,diagnosis,stage,help,group,data_source,date_form,requestid,sent,referer,form_name,city_2,city_score,region,coords,activity,loc_name,local,local_score,localization,stage_ill,stage_treatment,stage_score
0,User_01_71,Email_0071,tel_number_0071,1980-10-05 00:00:00,"Антрацит, ЛНР",Педагог,"Рак печени, 4 ст, эпителиоидная гемангоэндотел...","Химиотерапия, 5 курс",Консультацию по моему диагнощу,#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2021-12-10 00:00:00,3284862:2805026861,2021-12-10 00:40:19,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Образование, наука","печени, 4 ст, эпителиоидная гемангоэндотелиома",Лимфома,64,Злокачественные лимфомы,4,Химиотерапия,90
1,User_01_133,Email_0133,tel_number_0133,1982-04-03 00:00:00,Житомир Украина,Предприниматель,"РМЖ, трипл с мутацией, стадия 4",Химиотерапия,"Опыт лечения, Советы для облегчения побочен, С...",#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2022-01-06 00:00:00,3284862:2918786041,2022-01-06 23:32:25,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,77,Не указано,Не указано,"Предприниматель, самозанятый","РМЖ, трипл с мутацией, стадия 4",С 97,86,Множественная локализация,4,Химиотерапия,100
2,User_01_137,Email_0137,tel_number_0137,1978-11-06 00:00:00,Гомель Белорусия,Бухгалтер,РМЖ 4 стадия,"Химиотераия, операция.",Не указано,#ЛовлюЭмоции; Анонсы организации; Дари Эмоции,Интернет,2022-01-07 00:00:00,3284862:2918926121,2022-01-07 00:51:01,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Бухгалтерия, аудит",РМЖ 4 стадия,РМЖ,90,Молочная железа,4,Химиотерапия,83
3,User_01_147,Email_0147,tel_number_0147,1976-05-28 00:00:00,Минск Белорусия,Тренер,РМЖ 3 стадии,Химиотерапия,Информацию по лечению,Клуб “Рак Победим”; Анонсы организации,Интернет,2022-01-08 00:00:00,3284862:2921812661,2022-01-08 01:00:20,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Красота, фитнес, спорт",РМЖ 3 стадии,РМЖ,90,Молочная железа,3,Химиотерапия,100
4,User_01_149,Email_0149,tel_number_0149,1989-03-05 00:00:00,Астана Кахахстан,Временно не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",Таргетная терапия,Не указано,#ЛовлюЭмоции; Клуб “Рак Победим”; Дари Эмоции,Интернет,2021-01-08 00:00:00,3284862:2922682461,2022-01-08 12:14:36,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,Не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",опухоль кости,86,Кости и суставные хрящи,1,Таргетная терапия,100


In [ ]:
wards[wards.stage_treatment == 'CHECK']['stage'].value_counts()

Series([], Name: count, dtype: int64)

### Заявитель

Заявитель находится в столбце `diagnosis`, необходимо его выделить.

In [ ]:
# создам список для поиска
applicant_list = ['мама', 'мамы', 'маму', 'муж', 'супруг', 'доче', 'дочь', 'дочки', 'отец', 'отца', 'папы', 'папа', 'друг', 'сын', 'тетя', 'тети', 'болеет', 'сын', 'внук', 'сестр', 'брат']

# выделим заявителя
wards['applicant'] = wards['loc_name'].apply(lambda x: 'За третье лицо' if any(val in x or pd.Series(x).str.contains(val, regex=True, case=False).any() for val in applicant_list) \
  else 'За себя')

wards.head()

,name,email,phone_number,birthdate,city,fields_activity,diagnosis,stage,help,group,data_source,date_form,requestid,sent,referer,form_name,city_2,city_score,region,coords,activity,loc_name,local,local_score,localization,stage_ill,stage_treatment,stage_score,applicant
0,User_01_71,Email_0071,tel_number_0071,1980-10-05 00:00:00,"Антрацит, ЛНР",Педагог,"Рак печени, 4 ст, эпителиоидная гемангоэндотел...","Химиотерапия, 5 курс",Консультацию по моему диагнощу,#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2021-12-10 00:00:00,3284862:2805026861,2021-12-10 00:40:19,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Образование, наука","печени, 4 ст, эпителиоидная гемангоэндотелиома",Лимфома,64,Злокачественные лимфомы,4,Химиотерапия,90,За себя
1,User_01_133,Email_0133,tel_number_0133,1982-04-03 00:00:00,Житомир Украина,Предприниматель,"РМЖ, трипл с мутацией, стадия 4",Химиотерапия,"Опыт лечения, Советы для облегчения побочен, С...",#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2022-01-06 00:00:00,3284862:2918786041,2022-01-06 23:32:25,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,77,Не указано,Не указано,"Предприниматель, самозанятый","РМЖ, трипл с мутацией, стадия 4",С 97,86,Множественная локализация,4,Химиотерапия,100,За себя
2,User_01_137,Email_0137,tel_number_0137,1978-11-06 00:00:00,Гомель Белорусия,Бухгалтер,РМЖ 4 стадия,"Химиотераия, операция.",Не указано,#ЛовлюЭмоции; Анонсы организации; Дари Эмоции,Интернет,2022-01-07 00:00:00,3284862:2918926121,2022-01-07 00:51:01,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Бухгалтерия, аудит",РМЖ 4 стадия,РМЖ,90,Молочная железа,4,Химиотерапия,83,За себя
3,User_01_147,Email_0147,tel_number_0147,1976-05-28 00:00:00,Минск Белорусия,Тренер,РМЖ 3 стадии,Химиотерапия,Информацию по лечению,Клуб “Рак Победим”; Анонсы организации,Интернет,2022-01-08 00:00:00,3284862:2921812661,2022-01-08 01:00:20,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Красота, фитнес, спорт",РМЖ 3 стадии,РМЖ,90,Молочная железа,3,Химиотерапия,100,За себя
4,User_01_149,Email_0149,tel_number_0149,1989-03-05 00:00:00,Астана Кахахстан,Временно не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",Таргетная терапия,Не указано,#ЛовлюЭмоции; Клуб “Рак Победим”; Дари Эмоции,Интернет,2021-01-08 00:00:00,3284862:2922682461,2022-01-08 12:14:36,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,Не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",опухоль кости,86,Кости и суставные хрящи,1,Таргетная терапия,100,За себя


Помощь

In [ ]:
# посмотрим на столбец с помощью
wards.help.value_counts().to_frame().head(10)

,count
help,
Не указано,788
Психологическую,48
Общение,22
Психологическая,20
Информационную,18
Любую,13
Поддержку,12
Информация,12
Психологическая поддержка,9


In [ ]:
# создам словарь соответствий
help_dict = {
    'Не указано': ['Не указано'],
    'Материальная' : ['денеж', 'материал', 'подар', 'такси', 'оплат', 'гуманитар', 'стоит', 'трансфер', 'прожив', 'дорог', 'медикаме', 'реконструк', 'финанс', 'получи', 'терапи', 'жиль', 'обследов', 'бель', 'протез', 'рукав', 'билет', 'бюст', '300'],
    'Юридическая' : ['юридич', 'юрист', 'инвалиднос'],
    'Психологическая' : ['психол', 'эмоц', 'тревож', 'общен', 'морал', 'досуг', 'общат', 'страх', 'страш', 'говор', 'позитив', 'поддерж', 'реабилит', 'разобрат', 'переключит'],
    'Информационная' : ['консульт', 'инфо', 'совет', 'маршрут', 'обмен опыт', 'где', 'ормац', 'рекомендац', 'медицин', 'как ', 'что', 'мнени', 'вариант', 'новост', 'узна', 'знан', 'вопрос', 'врач', 'доктор', 'когда', 'можно', 'контакт', 'грамот', 'найти', 'про ', 'лимфо', 'эндокрин'],
    'Участие в мероприятиях': ['мероприят', 'очны', 'очно', 'встреч', 'нейрограф', 'йог', 'бассейн', 'танц', 'стилист', 'фото'],
    'Любая' : ['любая', 'любую', 'всесторон', 'всяк', 'любой', 'разн', 'всю']
    }

# создам новый столбец 'help_2'
wards['help_2'] = wards.help.apply(lambda x: next((key for key, values in help_dict.items() if any(pd.Series(x).astype(str).str.contains(value, regex=True, case=False).any() for value in (values))), 'Другая'))

wards[['help', 'help_2']][:10]

,help,help_2
0,Консультацию по моему диагнощу,Информационная
1,"Опыт лечения, Советы для облегчения побочен, С...",Информационная
2,Не указано,Не указано
3,Информацию по лечению,Информационная
4,Не указано,Не указано
5,Не указано,Не указано
6,Не указано,Не указано
7,Больше информации о питании и обследование,Материальная
8,Общение и внимание,Психологическая
9,Психологическую,Психологическая


### Проверю, что получилось

In [ ]:
wards.head(5)

,name,email,phone_number,birthdate,city,fields_activity,diagnosis,stage,help,group,data_source,date_form,requestid,sent,referer,form_name,city_2,city_score,region,coords,activity,loc_name,local,local_score,localization,stage_ill,stage_treatment,stage_score,applicant,help_2
0,User_01_71,Email_0071,tel_number_0071,1980-10-05 00:00:00,"Антрацит, ЛНР",Педагог,"Рак печени, 4 ст, эпителиоидная гемангоэндотел...","Химиотерапия, 5 курс",Консультацию по моему диагнощу,#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2021-12-10 00:00:00,3284862:2805026861,2021-12-10 00:40:19,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Образование, наука","печени, 4 ст, эпителиоидная гемангоэндотелиома",Лимфома,64,Злокачественные лимфомы,4,Химиотерапия,90,За себя,Информационная
1,User_01_133,Email_0133,tel_number_0133,1982-04-03 00:00:00,Житомир Украина,Предприниматель,"РМЖ, трипл с мутацией, стадия 4",Химиотерапия,"Опыт лечения, Советы для облегчения побочен, С...",#ЛовлюЭмоции; Клуб “Рак Победим”; Анонсы орган...,Знакомые,2022-01-06 00:00:00,3284862:2918786041,2022-01-06 23:32:25,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,77,Не указано,Не указано,"Предприниматель, самозанятый","РМЖ, трипл с мутацией, стадия 4",С 97,86,Множественная локализация,4,Химиотерапия,100,За себя,Информационная
2,User_01_137,Email_0137,tel_number_0137,1978-11-06 00:00:00,Гомель Белорусия,Бухгалтер,РМЖ 4 стадия,"Химиотераия, операция.",Не указано,#ЛовлюЭмоции; Анонсы организации; Дари Эмоции,Интернет,2022-01-07 00:00:00,3284862:2918926121,2022-01-07 00:51:01,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Бухгалтерия, аудит",РМЖ 4 стадия,РМЖ,90,Молочная железа,4,Химиотерапия,83,За себя,Не указано
3,User_01_147,Email_0147,tel_number_0147,1976-05-28 00:00:00,Минск Белорусия,Тренер,РМЖ 3 стадии,Химиотерапия,Информацию по лечению,Клуб “Рак Победим”; Анонсы организации,Интернет,2022-01-08 00:00:00,3284862:2921812661,2022-01-08 01:00:20,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,"Красота, фитнес, спорт",РМЖ 3 стадии,РМЖ,90,Молочная железа,3,Химиотерапия,100,За себя,Информационная
4,User_01_149,Email_0149,tel_number_0149,1989-03-05 00:00:00,Астана Кахахстан,Временно не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",Таргетная терапия,Не указано,#ЛовлюЭмоции; Клуб “Рак Победим”; Дари Эмоции,Интернет,2021-01-08 00:00:00,3284862:2922682461,2022-01-08 12:14:36,http://rak-pobedim.com/anketa,Анкета участника,Зарубежье,68,Не указано,Не указано,Не работаю,"РМЖ 1 ст, прогрессирование в кости, через 5 лет",опухоль кости,86,Кости и суставные хрящи,1,Таргетная терапия,100,За себя,Не указано


In [ ]:
# уникальные значения
for i in wards.columns:
  display(f'Количество уникальных значений в столбце {i}: {wards[i].nunique()}')
  display(wards[i].value_counts()[:10])
  print("")

'Количество уникальных значений в столбце name: 1568'

name
User_01_71      1
User_01_1030    1
User_01_1028    1
User_01_1026    1
User_01_1025    1
User_01_1024    1
User_01_1023    1
User_01_1022    1
User_01_1021    1
User_01_1020    1
Name: count, dtype: int64

'Количество уникальных значений в столбце email: 1424'

email
Не указано    145
Email_0071      1
Email_0992      1
Email_1003      1
Email_1002      1
Email_1001      1
Email_0998      1
Email_0997      1
Email_0996      1
Email_0995      1
Name: count, dtype: int64

'Количество уникальных значений в столбце phone_number: 1568'

...

stage_score
100    743
90     157
45      77
83      51
76      49
60      43
72      36
86      33
92      26
68      26
Name: count, dtype: int64

'Количество уникальных значений в столбце applicant: 2'

applicant
За себя           1492
За третье лицо      76
Name: count, dtype: int64

'Количество уникальных значений в столбце help_2: 8'

help_2
Не указано                788
Психологическая           387
Информационная            200
Материальная               74
Другая                     53
Любая                      31
Юридическая                23
Участие в мероприятиях     12
Name: count, dtype: int64